In [ ]:
import pandas as pd
import shutil
import yaml

from zephir.utils.utils import *

%matplotlib inline
from tqdm.notebook import tqdm

In [ ]:
# path to Zephir dataset
dataset = Path('.')

# path to DeepLabCut project directory
path_to_dlc_project = Path('.')

In [ ]:
with open(str(path_to_dlc_project / 'config.yaml'), 'r') as cfg:
    config = yaml.safe_load(cfg)
bodyparts = config['bodyparts']

shape_n = len(bodyparts)

metadata = get_metadata(dataset)
shape_t = metadata['shape_t']
img_shape = np.array([metadata['shape_x'], metadata['shape_y'], 1])

provenance = np.array([[bytes(provenance)] * shape_n] * shape_t)

In [ ]:
csv = pd.read_csv(str(dataset / 'data.csv'), header=[1, 2], index_col=0)
df = pd.DataFrame()
for k in csv:
    df[k] = csv[k]

t_list = df[('bodyparts', 'coords')]

In [ ]:
for t in tqdm(t_list, desc='Converting DLC results', unit='frame'):
    xyz_dlc = np.zeros((len(worldline_id), 3))
    for j, part in enumerate(dlc_parts):
        xyz = np.stack([(df[(part, 'x')][t]).astype(float),
                        (df[(part, 'y')][t]).astype(float),
                        0],
                       axis=-1)
        xyz_dlc[j] = xyz
    rms_dlc = np.mean(np.linalg.norm(xyz_target[i, :, :2] - xyz_dlc[:, :2], axis=-1))
    rms_dlc_avg += rms_dlc / len(t_target)
    m_dlc[i] = xyz_dlc[:, :2]

In [ ]:
if not (dataset / 'backup').is_dir():
    Path.mkdir(dataset / 'backup')
now = datetime.datetime.now()
now_ = now.strftime("%m_%d_%Y_%H_%M")
shutil.copy(dataset / 'annotations.h5',
            dataset / 'backup' / f'annotations_{now_}.h5')

In [ ]:
# saving result to .h5
print('\nCompiling and saving results to file...')
xyz_pd = np.concatenate(
    (np.repeat(np.arange(shape_t), shape_n)[:, None],
     results.reshape((-1, 3)) / 2.0 + 0.5,
     np.tile(np.arange(shape_n), shape_t)[:, None],
     np.repeat(np.arange(shape_t), shape_n)[:, None],
     provenance.reshape((-1, 1))),
    axis=-1
)

In [ ]:
f = h5py.File(dataset / 'annotations.h5', mode='w')

data = np.array(list(range(1, xyz_pd.shape[0] + 1)), dtype=np.uint32)
f.create_dataset('id', shape=(xyz_pd.shape[0],), dtype=np.uint32, data=data)

columns = {
    't_idx': np.uint32,
    'x': np.float32,
    'y': np.float32,
    'z': np.float32,
    'worldline_id': np.uint32,
    'parent_id': np.uint32,
    'provenance': np.dtype("S4"),
}

for i, c in enumerate(columns.keys()):
    if c == 'provenance':
        data = np.array(xyz_pd[:, i], dtype=columns[c])
    else:
        data = np.array(xyz_pd[:, i].astype(np.float32), dtype=columns[c])
    f.create_dataset(c, shape=(xyz_pd.shape[0],), dtype=columns[c], data=data)

f.close()